In [21]:
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()

movie_link = 'https://moviesjoytv.to/tv/its-in-the-game-madden-nfl-117676'

driver.get(movie_link)

movie = {
  'title': '',
  'src': movie_link,
  'releasedAt': '',
  'description': '',
  'poster': '',
  'duration': '',
  'seasons': -1,
  'imdbRating': '',
  'genres': [],
  'countries': [],
  'productions': [],
  'casts': [],
  'episodes': []
}

poster_section = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//div[@class="film-poster mb-2"]'))
)
movie['poster'] = poster_section.find_element(By.TAG_NAME, 'img').get_attribute('src')
movie_details_section = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//div[@class="dp-i-c-right"]'))
)
movie['title'] = movie_details_section.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a').text.strip()
movie['imdbRating'] = movie_details_section.find_element(By.XPATH, '//span[@class="item mr-2"]').find_element(By.TAG_NAME, 'button').text.replace('IMDB:', '').strip()
movie['description'] = movie_details_section.find_element(By.XPATH, '//div[@class="description"]').text.strip()

row_section = movie_details_section.find_element(By.XPATH, '//div[@class="row"]').find_elements(By.XPATH, '//div[@class="row-line"]')
movie['releasedAt'] = row_section[0].text.replace('Released:', '').strip()
movie['genres'] = [a_tag.text.strip() for a_tag in row_section[1].find_elements(By.TAG_NAME, 'a')]
movie['casts'] = [a_tag.text.strip() for a_tag in row_section[2].find_elements(By.TAG_NAME, 'a')]
movie['duration'] = row_section[3].text.replace('Duration:', '').strip()
movie['countries'] = [a_tag.text.strip() for a_tag in row_section[4].find_elements(By.TAG_NAME, 'a')]
movie['productions'] = [a_tag.text.strip() for a_tag in row_section[5].find_elements(By.TAG_NAME, 'a')]

episode_section = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//div[@id="content-episodes"]'))
)

episode_menu = episode_section.find_element(By.XPATH, '//div[@class="slt-seasons-dropdown"]')
episode_menu_childern = episode_menu.find_elements(By.XPATH, './*[1]')
driver.execute_script("arguments[0].click();", episode_menu_childern[0])
time.sleep(2)

# seasons = WebDriverWait(driver, 10).until(
#   EC.presence_of_element_located((By.XPATH, '//div[@class="dropdown-menu dropdown-menu-new"]'))
# ).find_elements(By.TAG_NAME, 'a')

seasons = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//*[@id="content-episodes"]/div/div[2]/div[1]/div[1]/div/div'))
).find_elements(By.TAG_NAME, 'a')

for season in seasons:
  driver.execute_script("arguments[0].click();", season)
  time.sleep(1)
  # episodes_per_season = WebDriverWait(driver, 10).until(
  #   EC.presence_of_element_located((By.XPATH, '//*[@id="content-episodes"]/div/div[2]/div[1]/div[3]/div'))
  # )

full_rendered_episodes = WebDriverWait(driver, 10).until(
  EC.presence_of_element_located((By.XPATH, '//*[@id="content-episodes"]/div/div[2]/div[1]/div[3]/div'))
).find_elements(By.XPATH, './div')

movie['seasons'] = len(full_rendered_episodes)

for season_number, div in enumerate(full_rendered_episodes):
  episodes_per_season = div.find_elements(By.TAG_NAME, 'a')
  for episode_number, episode in enumerate(episodes_per_season):
    episode_details = {
      'season': season_number + 1,
      'number': episode_number + 1,
      'title': '',
      'link': ''
    }
    episode_details['title'] = episode.get_attribute('title').split(':')[1].strip()
    movie['episodes'].append(episode_details)

with open('DUMMY.json', 'w', encoding='utf-8') as f:
  json.dump(movie, f, ensure_ascii=False, indent=2)